# Introduction
We will try some stunts to use the best possible way to use asyncio with tqdm on IBKR in this notebook

# Running asyncio in Jupyter notebook

In a normal <i>xxxx.py</i> file, the following code should run. It however fails in Jupyter Notebook with a `RuntimeError`

In [ ]:
import asyncio

async def main():
    print(1)

asyncio.run(main())


The RuntimeError happens because Jupyter already has an event loop running. 

## Solution # 1
One can rewrite the code by `await`ing the main function rather than `asyncio.run`ing it. This requires iPython ver 7+, iPykernel ver 5+. 

In [ ]:
import asyncio

async def main():
    print(1)

await main()

## Solution # 2

Another alternative is to `nest` a new asyncio loop to Jupyter's own asyncio loop using the nifty `nest_asyncio` utility.

This - of course - requires installation of the utility and `apply()`ing it.

> pip install nest_asyncio

In [ ]:
import asyncio

import nest_asyncio
nest_asyncio.apply()

async def main():
    print(1)

asyncio.run(main())

# Using asyncio with tqdm progress bars

One can use asyncio with tqdm for progress bars as follows:

In [ ]:
import asyncio
import random

import tqdm


async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        await asyncio.sleep(random.random())
        f *= i
    return f"Task {name}: factorial {number} = {f}"

async def tq(flen):
    for _ in tqdm.tqdm(range(flen)):
        await asyncio.sleep(0.1)


async def main():

    flist = [factorial("A", 2),
             factorial("B", 3),
             factorial("C", 4)]

    pbar = tqdm.tqdm(total=len(flist), position=0, ncols=90)
    for f in asyncio.as_completed(flist):
        value = await f
        pbar.set_description(desc=value, refresh=True)
        tqdm.tqdm.write(value)
        pbar.update()
    
    pbar.close()

if __name__ == '__main__':

    # if nest_asyncio is applied you can use asyncio.run below
    # asyncio.run(main())

    # if nest_asyncio is not there, you can run the asyncio using await main()
    await main()


# IBKR with asyncio in native Jupyter

Now let us take a look at how asyncio works on IBKR

We will get OHLC for two contracts - one a stock and aonther an index

**Note**: The program behaves strangely in Jupyter. Most probably this is due to clash of loops

- Sometimes it gives `NameError: name 'IB' is not defined`
- Other times it gives an error and suddenly works again

You may need to restart the kernel


In [ ]:
import sys
import pandas as pd
import asyncio
import yaml

from ib_insync import IB, util

util.startLoop()

MARKET = 'SNP'

ROOT_PATH = r'C:/Users/User/Documents/Business/Projects/iboop/'
FSPATH = ROOT_PATH+r'data/'+MARKET.lower()+'/'

sys.path.insert(0, ROOT_PATH)

# Variables initialization
with open(ROOT_PATH+'var.yml') as f:
    data=yaml.safe_load(f)

HOST = data["COMMON"]["HOST"]
PORT = data[MARKET.upper()]["PORT"]
CID = data["COMMON"]["CID"]

# Get two contracts
df_unds = pd.read_pickle(FSPATH+'_df_unds.pkl')
contracts = df_unds.groupby('ctype').head(1).contract

# Make a single async function
async def get_ohlc(ib, c):

    ohlc = await ib.reqHistoricalDataAsync(
        contract=c,
        endDateTime="",
        durationStr="365 D",
        barSizeSetting="1 day",
        whatToShow="Trades",
        useRTH=True
    )

    # reverse sort to have latest date on top
    df_ohlc = util.df(ohlc).sort_index(ascending=False).reset_index(drop=True)

    df_ohlc.insert(0, 'symbol', c.symbol)

    return df_ohlc

Here will try to gather the task using asyncio.gather and run it using await - the most basic in Jupyter

In [ ]:
with IB().connect(host=HOST, port=PORT, clientId=CID) as ib:
    tasks = asyncio.gather(*[get_ohlc(ib, c) for c in contracts])
    ohlcs = await tasks
    ib.sleep(1)

df = pd.concat(ohlcs)
df.head()

# Running an external asynchronous .py file in Jupyter
Let us now try to run *async_ib.py* file which is outside jupyter by importing it.

In [1]:
%reset
import sys
import async_ib

from ib_insync import util
util.startLoop()

MARKET = 'SNP'

ROOT_PATH = r'C:/Users/User/Documents/Business/Projects/iboop/'
sys.path.insert(0, ROOT_PATH)

async_ib.get_2_ohlcs().groupby('symbol').head()


Nothing done.


,symbol,date,open,high,low,close,volume,average,barCount
0,MXEA,2020-04-08,1578.21,1582.45,1569.64,1582.20,0,0.0000,1355
1,MXEA,2020-04-07,1591.83,1591.83,1568.63,1580.02,0,0.0000,1405
2,MXEA,2020-04-06,1535.01,1543.09,1532.25,1542.18,0,0.0000,1369
3,MXEA,2020-04-03,1491.36,1493.85,1483.71,1488.51,0,0.0000,1366
4,MXEA,2020-04-02,1495.89,1516.70,1492.69,1504.65,0,0.0000,1433
0,AAPL,2020-04-08,262.74,267.37,261.23,266.07,293136,264.5680,144484
1,AAPL,2020-04-07,270.80,271.70,259.00,259.43,389906,264.8230,185526
2,AAPL,2020-04-06,250.87,263.11,249.38,262.47,351552,255.3605,183674
3,AAPL,2020-04-03,242.90,245.70,238.97,241.41,242840,241.8235,121664
4,AAPL,2020-04-02,240.24,245.15,236.90,244.93,320620,241.2630,165947


It just works beautifully! (If the cell is run by itself)